In [6]:
import pandas as pd

df_source = pd.read_excel('~/Downloads/english_words_12931.xlsx')
df_source['音标美'].fillna('', inplace=True)
df_source['音标英'].fillna('', inplace=True)
df_source.head()

,单词,音标美,音标英,意思
0,a cushion of,,,网络一层
1,a few,,,na.几个；〔反语〕很多；〈俚〉一点点 网络意思：一些；有些；少量
2,a good deal,,,na.“a deal”的变体 网络意思：许多；大量；诸多
3,a great deal,,,na.很多 网络意思：大量；许多；大量的
4,a little,,,n.一点 网络意思：一些；少许；有点


In [18]:
from pymongo import MongoClient

def import_data(df_source):
    CONNECTION_STRING = 'mongodb://localhost:27017'
    BATCH_INSERT_COUNT = 30

    client = MongoClient(CONNECTION_STRING)

    english_words_v1 = client['quiz-book']['english_words_v1']

    records_to_insert = []
    for i, row in df_source.iterrows():
        record = {
            'en': row['word'],
            'phonetic_us': row['phonetic_us'].replace('美', '').strip(),
            'phonetic_br': row['phonetic_br'].replace('英', '').strip(),
            'zh': row['translation']
        }

        records_to_insert.append(record)

        if len(records_to_insert) >= BATCH_INSERT_COUNT:
            english_words_v1.insert_many(records_to_insert)
            records_to_insert = []

    if len(records_to_insert) > 0:
        english_words_v1.insert_many(records_to_insert)

In [21]:
import pandas as pd

ds_left = pd.read_excel('~/Downloads/EnWords.xlsx', sheet_name='words')
ds_right = pd.read_excel('~/Downloads/EnWords.xlsx', sheet_name='phonetic')
ds_right.rename({'Word': 'word'}, axis=1, inplace=True)
ds_right.rename({'美式音标': 'phonetic_us'}, axis=1, inplace=True)
ds_right.rename({'英式音标': 'phonetic_br'}, axis=1, inplace=True)
ds_right.drop(columns=['Mdict_ETDict'], inplace=True)

# ds_right.head()

ds_result = ds_left.merge(ds_right, on='word', how='left')
ds_result.fillna(value='', inplace=True)
ds_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105416 entries, 0 to 105415
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   word         105416 non-null  object
 1   translation  105416 non-null  object
 2   phonetic_us  105416 non-null  object
 3   phonetic_br  105416 non-null  object
dtypes: object(4)
memory usage: 4.0+ MB


In [22]:
import_data(ds_result)